In [1]:
import numpy as np
import torchvision.models as models

In [2]:
# load and view pretrained SqueezeNet v1.1
# https://github.com/DeepScale/SqueezeNet
squeezenet = models.squeezenet1_1(pretrained=True)
squeezenet

SqueezeNet (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Fire (
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU (inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU (inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU (inplace)
    )
    (4): Fire (
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU (inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU (inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU (inplace)
    )
    (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1,

In [3]:
# get all model parameters
params = list(squeezenet.named_parameters())

In [4]:
def get_weights(name):
    return [p for p in params if name in p[0]][0][1]

In [6]:
# layers renaming scheme
names_dict = {
    'features.0': 'conv1',
    '3.expand1x1': 'fire2/expand1x1', '3.expand3x3': 'fire2/expand3x3', 
    '3.squeeze': 'fire2/squeeze',
    '4.expand1x1': 'fire3/expand1x1', '4.expand3x3': 'fire3/expand3x3',
    '4.squeeze': 'fire3/squeeze',
    '6.expand1x1': 'fire4/expand1x1', '6.expand3x3': 'fire4/expand3x3',
    '6.squeeze': 'fire4/squeeze',
    '7.expand1x1': 'fire5/expand1x1', '7.expand3x3': 'fire5/expand3x3',
    '7.squeeze': 'fire5/squeeze',
    '9.expand1x1': 'fire6/expand1x1', '9.expand3x3': 'fire6/expand3x3',
    '9.squeeze': 'fire6/squeeze',
    '10.expand1x1': 'fire7/expand1x1', '10.expand3x3': 'fire7/expand3x3',
    '10.squeeze': 'fire7/squeeze',
    '11.expand1x1': 'fire8/expand1x1', '11.expand3x3': 'fire8/expand3x3',
    '11.squeeze': 'fire8/squeeze',
    '12.expand1x1': 'fire9/expand1x1', '12.expand3x3': 'fire9/expand3x3',
    '12.squeeze': 'fire9/squeeze',
    'classifier.1': 'conv10'
 }

In [7]:
weights = {}

In [11]:
# pytorch kernel shape: output, input, h, w
# keras kernel shape: h, w, input, output
# so I need to transpose

for k in names_dict:
    new_name = names_dict[k]
    weights[new_name] = [
        get_weights(k + '.weight').transpose(0, 2).transpose(1, 3).transpose(2, 3).data.numpy(),
        get_weights(k + '.bias').data.numpy()
    ]

In [12]:
np.save('weights.npy', weights)